### for starting run all cells consequentially

In [ ]:
#при первом запуске
#conda install -c conda-forge ipywidgets
import ipywidgets as widgets
from IPython.display import clear_output

In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import threading
import time

class IBapi(EWrapper, EClient):
    
    orders = {}
    
    def __init__(self):
        EClient.__init__(self, self)
        
    
    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)

    def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, 
                    lastFillPrice, clientId, whyHeld, mktCapPrice):
        print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 
              'remaining', remaining, 'lastFillPrice', lastFillPrice)
    
    def openOrder(self, orderId, contract, order, orderState):
        print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', 
              order.action, order.orderType, order.totalQuantity, orderState.status)
        
    def execDetails(self, reqId, contract, execution):
        print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, 
              execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)
      

    
def FX_order(symbol):
    contract = Contract()
    contract.symbol = symbol[:3]
    contract.secType = 'CASH'
    contract.exchange = 'IDEALPRO'
    contract.currency = symbol[3:]
    return contract

In [ ]:
def start(b):

    def create_order(order_type, quantity, price, action, parent_order = Order(), last = False):
    
        order = Order()
        order.orderType = order_type
        order.totalQuantity = quantity
        
        if (order_type == 'LMT'):
            order.lmtPrice = price
        else:
            order.auxPrice = price
            
        order.action = action
        
        order.orderId = app.nextorderId
        app.nextorderId += 1
        
        order.parentId = parent_order.orderId
            
        if (last != True):
            order.transmit = False   
        
        return order
    
    

    app = IBapi()
    
    #paste your connection details here 
    app.connect('127.0.0.1', 7497, 123)
    
    app.nextorderId = None
    
    #Start the socket in a thread
    api_thread = threading.Thread(target=app.run, daemon=True)
    api_thread.start()
    
    #Check if the API is connected via orderid
    while True:
        if isinstance(app.nextorderId, int):
            print('connected')
            print()
            break
        else:
            print('waiting for connection')
            time.sleep(1)
    
            
    b1 = (wStopLoss.value == False) and (wTakeProfit.value == False)      
    
    #Create order object
    order = create_order(wType1.value, wQuantity.value, wPrice.value, wType2.value, last = b1)
    app.placeOrder(order.orderId, FX_order('EURUSD'), order)
    
    
    b2 = (wTakeProfit.value == False)
    operation_type = 'SELL' if wType2.value == 'BUY' else 'BUY'
    
    #Create stop loss order object
    if (wStopLoss.value == True):
        sl_order = create_order('STP', 100, '1.05', operation_type, order, last = b2)
        
        app.placeOrder(sl_order.orderId, FX_order('EURUSD'), sl_order)
    
    
    #Create take profit order object
    if (wTakeProfit.value == True):
        tp_order = create_order('LMT', 100, '1.15', operation_type, order, last = True)
        
        app.placeOrder(tp_order.orderId, FX_order('EURUSD'), tp_order)
    
        app.cancelOrder(tp_order.orderId)
    
    app.disconnect()
    
    

In [ ]:
wType1 = widgets.Dropdown(
    options=['LMT', 'MKT'],
    value='LMT',
    description='Type1:',
    disabled=False,
)

wType2 = widgets.Dropdown(
    options=['BUY', 'SELL'],
    value='BUY',
    description='Type2:',
    disabled=False,
)


wPrice = widgets.BoundedFloatText(
    value=1.1,
    min=0,
    max=10.0,
    step=0.01,
    description='Price:',
    disabled=False
)

wQuantity = widgets.IntText(
    value=10,
    description='Total quantity:',
    disabled=False
)

wStopLoss = widgets.Checkbox(
    value=False,
    description='Stop loss',
    disabled=False
)

wTakeProfit = widgets.Checkbox(
    value=False,
    description='Take profit',
    disabled=False
)


wButton = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)


wPriceSt = widgets.BoundedFloatText(
    value=1.05,
    min=0,
    max=10.0,
    step=0.01,
    description='Price:',
    disabled=False
)

wQuantitySt = widgets.IntText(
    value=10,
    description='Total quantity:',
    disabled=False
)

wPriceTp = widgets.BoundedFloatText(
    value=1.15,
    min=0,
    max=10.0,
    step=0.01,
    description='Price:',
    disabled=False
)

wQuantityTp = widgets.IntText(
    value=10,
    description='Total quantity:',
    disabled=False
)


wButton.on_click(start)

In [ ]:
def panel():
    clear_output(wait=True)
    
    display(wType1, wType2)
    
    if (wType1.value == 'LMT'):
        display(wPrice)
        
    display(wQuantity, wStopLoss)
    
    if (wStopLoss.value == True):
        display(wPriceSt, wQuantitySt)
    
    display(wTakeProfit)
    
    if (wTakeProfit.value == True):
        display(wPriceTp, wQuantityTp)
        
    display(wButton)

In [ ]:
#for realtime widgets tracking
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [ ]:
async def fSt():
    while True:
        await wait_for_change(wStopLoss, 'value')
        panel()
        
async def fTp():
    while True:
        await wait_for_change(wTakeProfit, 'value')
        panel()
        
async def fType1():
    while True:
        await wait_for_change(wType1, 'value')
        panel()

In [ ]:
#start tracking widgets
asyncio.ensure_future(fSt())
asyncio.ensure_future(fTp())
asyncio.ensure_future(fType1())

In [ ]:
#main panel
#make ur choice here
panel()